In [1]:
import syft as sy


kj/filesystem-disk-unix.c++:1703: warning: PWD environment variable doesn't match current directory; pwd = /home/teo/OpenMined/PySyft


In [2]:
node = sy.orchestra.launch(name="test-jobs", dev_mode=True, reset=True, create_producer=True, queue_port=5556, n_consumers=3)
client = node.login(email="info@openmined.org", password="changethis")

Staging Protocol Changes...
Timeout elapsed after 30 seconds while trying to acquiring lock.
FAILED TO LOCK
Logged into <test-jobs: High side Domain> as <info@openmined.org>


SyftWarning: You are using a default password. Please change the password using `[your_client].me.set_password([new_password])`.

In [3]:
x = sy.ActionObject.from_obj([1,2,3])
x_ptr = x.send(client)

In [4]:
@sy.syft_function()
def process_batch(batch):
    # takes 30 hours normally
    print(f"starting batch {batch}")
    from time import sleep
    import random
    sleep(random.random()*5)
    print("done")
    return batch + 1

SyftSuccess: Syft function 'process_batch' successfully created. To add a code request, please create a project using `project = syft.Project(...)`, then use command `project.create_code_request`.

In [5]:
client.code.submit(process_batch)

SyftSuccess: User Code Submitted

In [6]:
@sy.syft_function()
def aggregate_job(job_results):
    results = [job.resolve.get() for job in job_results]
    print([(res, type(res))for res in results])
    return sum(results)

SyftSuccess: Syft function 'aggregate_job' successfully created. To add a code request, please create a project using `project = syft.Project(...)`, then use command `project.create_code_request`.

In [7]:
client.code.submit(aggregate_job)

SyftSuccess: User Code Submitted

In [8]:
@sy.syft_function_single_use(x=x_ptr)
def process_all(domain, x):
    jobs = []
    print("Launching jobs")
    for elem in x:
        # We inject a domain object in the scope
        batch_job = domain.launch_job(process_batch, batch=elem)
        jobs.append(batch_job)
    print("starting aggregation")
    print("Done")
    return domain.launch_job(aggregate_job, job_results=jobs)

SyftSuccess: Syft function 'process_all' successfully created. To add a code request, please create a project using `project = syft.Project(...)`, then use command `project.create_code_request`.

In [9]:
client.code.request_code_execution(process_all)
client.requests[-1].approve()

Request approved for domain test-jobs


SyftSuccess: Request 8e905e074c434a27a4de89eee7172976 changes applied

In [10]:
from time import sleep
job = client.code.process_all(x=x_ptr, blocking=False)
sleep(10)

ACTION: [] {'x': <LineageID: cc077e8ae62c497884786c1afeeccdf6 - 339001915512658413>}


True True


20/11/23 19:12:37 FUNCTION LOG (eba77a306c6a4a89a50ced85c63de4ac): Launching jobs
ACTION: [] {'batch': <LineageID: 9748305864244a1b80e0c39785fd8f00 - 1955202543397101569>}


True True
True True


ACTION: [] {'batch': <LineageID: 9bc9755ed64e40e698a9660f1e223525 - 1654247154800409728>}
20/11/23 19:12:39 FUNCTION LOG (eba77a306c6a4a89a50ced85c63de4ac): starting aggregation
20/11/23 19:12:39 FUNCTION LOG (eba77a306c6a4a89a50ced85c63de4ac): Done
20/11/23 19:12:39 FUNCTION LOG (affb8d3601914bf99bb4175539ed2bf7): starting batch 2
20/11/23 19:12:39 FUNCTION LOG (06d1d76b554b4cfeaa684661ff458667): starting batch 1
ACTION: [] {'batch': <LineageID: ff11536ced7d467680e5effd2a2b70a4 - 1833118312470783127>}
ACTION: [] {'job_results': <LineageID: 66bc925df3094958aa55d687d41235d0 - 16441510289244335>}


...

20/11/23 19:12:41 FUNCTION LOG (06d1d76b554b4cfeaa684661ff458667): done
ACTION: [] {'job_results': <LineageID: 66bc925df3094958aa55d687d41235d0 - 16441510289244335>}


True False


20/11/23 19:12:42 FUNCTION LOG (affb8d3601914bf99bb4175539ed2bf7): done
ACTION: [] {'job_results': <LineageID: 66bc925df3094958aa55d687d41235d0 - 16441510289244335>}


True True


20/11/23 19:12:43 FUNCTION LOG (9ae2bb72d56545d486f253e375d3ef0a): [(2, <class 'int'>), (3, <class 'int'>), (4, <class 'int'>)]


In [11]:
result = job.resolve.get().resolve.get()
result

9